#Алгоритм обработки писем

1. Берем заголовок (регулярками смотрим, есть ли там ФИО или ФИ) (name);
2. Берем абзацы текста:
- удаляем тэги в конце;
- чистим от ненужных символов;
- проверяем первый абзац на наличие ключей - чтобы определить примечания составителей (метаданные, meta и name, если нет в заголовке);
- находим регулярками в абзацах даты;
- даты записываем в нужном формате - для xml (date);
- из дат вытскиваем год (year)
- находим текст письма/разделяем текст по датам на несколько писем (если их несколько на странице).
3. Записываем каждое письмо в xml-файл


In [0]:
from bs4 import BeautifulSoup
import requests
import lxml.etree as etree #модуль для создания xml файлов
import re
import lxml.html

In [0]:
#функция, создающая дату для xml-файла (тэг <date>)

def make_date(date):
  date = date.replace('г.', '')
  date = date.strip()
  date = date.split(sep = ' ')
  date.reverse()
  months = {'январь' : '01', 'января': '01', 'февраль':'02', 'февраля':'02', 'март':'03', 'марта':'03', 'апрель':'04', 'апреля':'04', 'май':'05', 'мая':'05', 'июнь':'06', 'июня':'06',
            'июль':'07', 'июля':'07', 'август':'08', 'августа':'08', 'сентябрь':'09', 'сентября':'09', 'октябрь':'10', 'октября':'10', 'ноябрь':'11', 'ноября':'11', 'декабрь':'12', 'декабря':'12'}
  for i in date:
    if i in months.keys():
      date[date.index(i)] = months[i]
  if len(date) == 1:
    date.append('-')
    date.append('-')
  elif len(date) == 2:
    date.append('-')
  date = '-'.join(date)
  return date

In [0]:
#функция, забирающая текст одного письма на странице

def make_1let (text):
  meta_words = ['родился', 'погиб', 'окончил', 'адресовано', 'адресованы', 'уроженец', 'работал', 'награждён', 'письмо']
  meta = None
  for i in text:
    if any(substr in i.lower() for substr in meta_words):
      if text.index(i) == 1:
        meta = i
      letter = text[1:]
    else:
      letter = text
  return letter

In [0]:
#функция, переупаковывающая html в TEI xml

def make_TEI(name, year, date, date2, meta, letter):
  tei = etree.Element('TEI', xmlns = "http://www.tei-c.org/ns/1.0")
  teiHeader = etree.SubElement(tei, 'teiHeader')
  fileDesc = etree.SubElement(teiHeader, 'fileDesc')
  profileDesc = etree.SubElement(teiHeader, 'profileDesc')
  textDesc = etree.SubElement(profileDesc, 'textDesc')
  date = etree.SubElement(textDesc, 'date', when = date2).text = date
  meta = etree.SubElement(textDesc, 'meta').text = meta
  titleStmt = etree.SubElement(fileDesc, 'titleStmt')
  year = etree.SubElement(titleStmt, 'year').text = year        
  author = etree.SubElement(titleStmt, 'author').text = name
  publicationStmt = etree.SubElement(fileDesc, 'publicationStmt')
  p = etree.SubElement(publicationStmt, 'p').text = 'Издательство "Книга", Краснодар https://www.pismasfronta.com/'        
  sourceDesc = etree.SubElement(fileDesc, 'sourceDesc')
  p = etree.SubElement(sourceDesc, 'p').text = url
  text = etree.SubElement(tei, 'text')
  body = etree.SubElement(text, 'body')   
  p = etree.SubElement(body, 'p').text = letter
  tree = etree.ElementTree(tei)
  return tree

In [0]:
many_letters = []
no_text= []
made_urls = []
made_urls_2 = []

#в списке many_letters будут ссылки на страницы, где не одно письмо, а несколько. Его будем обрабатывать отдельно
#в списке no_text будут сохраняться ссылки, где код не нашел текста. Их обработаем вручную
#списки made_urls и made_urls_2 нужны для проверки (на всякий случай на дублирование страниц)

In [0]:
#прорабатываем отдельно каждый файл со ссылками, отсортированными на прудыдущем этапе:
#'pisma_good_links1.txt'
#'pisma_good_links2.txt'
#'pisma_good_links0.txt' 
#'pisma_error_links.txt'

#сюда поочередно подставляем путь к файлам с отсортированными ссылками

with open('/content/drive/My Drive/ВКР/Отсортированные ссылки/pisma_good_links1.txt', 'r', encoding='utf-8') as file:
  urls = file.readlines()

In [0]:
# в переменной cnt - считается общее число обработанных писем. Она же используется для нумерации файлов. Ее нужно обновлять для каждого нового файла со ссылками.
cnt = 0
for url in urls:
  url = url.replace('\n', '')
  if url not in made_urls:
    made_urls.append(url)
    r = requests.get(url)
    page_source = r.text
    soup = BeautifulSoup(page_source, 'lxml')
    try:
      name = soup.find('strong').text
      print(name)
    #ищем имя в заголовке
      if re.findall(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+.+', name) or name == 'Автор неизвестен':
        name = name
      else:
        name = None
    except:
      name = None
      #забираем весь текст, складываем в список, убираем тэги в конце, чистим от знаков
    full_text = soup.find('div', {'class':'flex_display flex_vbox'})
    texts = []
    try:
      for sub_heading in full_text.find_all(['p','h1', 'h2', 'h3', 'h4', 'h5']):
        texts.append(sub_heading.text)
    except:
      texts.append(None)
      no_text.append(url)
    ind = None
    for i in texts:
      if i == 'Tags:':
        ind = texts.index(i)
    del texts[ind:]
    for i in texts:
      if i == '\xa0':
        del texts[texts.index(i)]
      i = i.replace('\xa0', ' ')
    for i in texts:
      if i == name:
        del texts[texts.index(i)]     
        #забираем подписи авторов и имя, если его не было в заголовке
    meta_words = ['родился', 'погиб', 'окончил', 'адресовано', 'адресованы', 'уроженец', 'работал', 'награждён', 'письмо']
    meta = None
    for i in texts:
      if any(substr in i.lower() for substr in meta_words):
        if texts.index(i) == 0:
          meta = i
          texts.remove(i)
    for i in texts:
      if name == None:
        try:
          if re.findall(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+.+', meta):
            name = re.findall(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+.+', meta)
            texts.remove(i)
        except:
          name == None
    try:
      if re.findall(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+.+', texts[0]):
        texts.remove(texts[0])
    except:
      continue 
    letter = []  
    reg = r'^.+\d{4}\s*г\.$'
    reg2 = r'\d{4}'
    reg3 = r'\d{4}\s*г\.$'
    data = []
    ind = None
    for i in texts:
      if re.findall(reg, i) or re.findall(reg3, i):
        data.append(i)
        ind = texts.index(i)
    if len(data) == 0:
      year = ['Без даты']
      date = 'Без даты'
      letter = make_1let(texts[0:])
    if len(data) == 1:
      date = data[0]
      year = re.findall(reg2, data[0])
      letter = make_1let(texts[ind:]) 
    if len(data) > 1:
      print('Много писем')
      many_letters.append(url)
    year = year[0]
    try:
      if letter[0] == date or letter[0] == year or letter[0] == '[Дата неизвестна]':
        del letter[0]
    except:
      continue
    letter = ' '.join(letter)
    letter = letter.replace('(', '\<!--')
    letter = letter.replace('[', '\<!--')
    letter = letter.replace(')', '\>')
    letter = letter.replace(']', '\>')
    letter = letter.replace('<…>', '')
    letter = letter.replace('<…>', '')
    if date != 'Без даты':
      date = date.strip()
      date2 = make_date(date)
    else:
      date2 = '----'
    tree = make_TEI(name, year, date, date2, meta, letter)
    print(tree)
    tree.write(f'/content/drive/My Drive/ВКР/corpus_ver2/letter{cnt}.xml', encoding = 'utf8', pretty_print = True, \
    xml_declaration = True)
    cnt+=1   

print(cnt)

In [0]:
# код для обработки many_letters
# не забудьте обновить переменную cnt тем числом, на котром закончиась нумерация в прошлом цикле

cnt = 635 
for url in many_letters:
  url = url.replace('\n', '')
  if url not in made_urls_2:
    made_urls_2.append(url)
    r = requests.get(url)
    page_source = r.text
    soup = BeautifulSoup(page_source, 'lxml')

  # ищем имя в заголовке
    try:
      name = soup.find('strong').text
      print(name)
      if re.findall(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+.+', name) or name == 'Автор неизвестен':
        name = name
      else:
        name = None
    except:
      name = None
  #забираем весь текст, складываем в список, убираем тэги в конце, чистим от знаков
    full_text = soup.find('div', {'class':'flex_display flex_vbox'})
    texts = []
    try:
      for sub_heading in full_text.find_all(['p','h1', 'h2', 'h3', 'h4', 'h5']):
        texts.append(sub_heading.text)
    except:
      texts.append(None)
      no_text.append(url)
    ind = None
    for i in texts:
      if i == 'Tags:':
        ind = texts.index(i)
    del texts[ind:]
    for i in texts:
      if i == '\xa0':
        del texts[texts.index(i)]
      i = i.replace('\xa0', ' ')
    for i in texts:
      if i == name:
        del texts[texts.index(i)]     
  #забираем подписи авторов и имя, если его не было в заголовке
    meta_words = ['родился', 'погиб', 'окончил', 'адресовано', 'адресованы', 'уроженец', 'работал', 'награждён', 'письмо']
    meta = None
    for i in texts:
      if any(substr in i.lower() for substr in meta_words):
        if texts.index(i) == 0:
          meta = i
          texts.remove(i)
    for i in texts:
      if name == None:
        try:
          if re.findall(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+.+', meta):
            name = re.findall(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+.+', meta)
            texts.remove(i)
        except:
          name == None
    try:
      if re.findall(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+.+', texts[0]):
        texts.remove(texts[0])
    except:
      continue 
    indexes = []
    reg = r'^.+\d{4}\s*г\.$'
    reg2 = r'\d{4}'
    reg3 = r'\d{4}\s*г\.$'
    letters = []
    for i in texts:
      if re.findall(reg, i) or re.findall(reg3, i):
        indexes.append(texts.index(i))
    try:  
      for i in range(len(indexes)+1):
        letter = texts[indexes[i]:indexes[i+1]]
        letters.append(letter)
    except IndexError:
      letter = texts[indexes[-1]:]
      letters.append(letter)
  #забираем дату для каждого письма

    for let in letters:
      date = 0
      date2 = 0
      year = 0
      try:
        if re.findall(reg, let[0]) or re.findall(reg2, let[0]) or re.findall(reg3, let[0]):
          date = let[0]
          date = date.strip()
          date2 = make_date(date)
          year = re.findall(reg2, date)
          year = year[0]
        else:
          date = 'Без даты'
          date2 = '----'
          year = 'Без даты' 
      except:
        date = 'Без даты'
        date2 = '----'
        year = 'Без даты' 
      try:
        if let[0] == date or let[0] == year or let[0] == '[Дата неизвестна]':
          del let[0]
      except:
        continue
      let = ' '.join(let)
      let = let.replace('(', '\<!--')
      let = let.replace('[', '\<!--')
      let = let.replace(')', '\>')
      let = let.replace(']', '\>')
      let = let.replace('<…>', '')
      let = let.replace('<…>', '')
      tree = make_TEI(name, year, date, date2, meta, let)
      print(tree)
      tree.write(f'/content/drive/My Drive/ВКР/corpus_ver2/letter{cnt}.xml', encoding = 'utf8', pretty_print = True, \
      xml_declaration = True)
      cnt+=1  

print(cnt)